In [10]:
# 1. CARGA DE LIBRERÍAS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.signal import periodogram
from sklearn.decomposition import PCA


In [33]:
# 2. CARGA DE DATOS
path_tsm = r'C:\Users\Lenovo\Desktop\MCD\2025-1\Estadistica\ae-oceanografia-golfo-cali\data\raw\Base de datos TSM consultoria estadística.xlsx'
path_chla = r'C:\Users\Lenovo\Desktop\MCD\2025-1\Estadistica\ae-oceanografia-golfo-cali\data\raw\Base de datos Chla consultoria estadística.xlsx'
path_coords = r'C:\Users\Lenovo\Desktop\MCD\2025-1\Estadistica\ae-oceanografia-golfo-cali\data\raw\Coordenadas zona costera occidental GC.csv'
path_oni = r'C:\Users\Lenovo\Desktop\MCD\2025-1\Estadistica\ae-oceanografia-golfo-cali\data\raw\oni-Cold & Warm Episodes by Season.csv - Sheet1.csv'

tsm_df = pd.read_excel(path_tsm)
chla_df = pd.read_excel(path_chla)
coords_df = pd.read_csv(path_coords, header=None)
oni_df_raw  = pd.read_csv(path_oni)


In [35]:
tsm_df.columns = tsm_df.columns.str.strip()
chla_df.columns = chla_df.columns.str.strip()

# Derretir (melt) a formato largo
tsm_long = tsm_df.melt(
    id_vars=["FECHA", "AÑO", "MES", "ESTACIÓN DEL AÑO", "EVENTO INTERANUAL"],
    value_vars=[col for col in tsm_df.columns if "Est" in col],
    var_name="estacion",
    value_name="TSM"
)

chla_long = chla_df.melt(
    id_vars=["FECHA", "AÑO", "MES", "ESTACIÓN DEL AÑO", "EVENTO INTERANUAL"],
    value_vars=[col for col in chla_df.columns if "Est" in col],
    var_name="estacion",
    value_name="Chla"
)

# Unir datasets por columnas comunes
data_long = pd.merge(tsm_long, chla_long, on=["FECHA", "AÑO", "MES", "ESTACIÓN DEL AÑO", "EVENTO INTERANUAL", "estacion"], how="outer")

# Convertir FECHA a datetime por si no está
data_long["FECHA"] = pd.to_datetime(data_long["FECHA"])

# Vista previa
data_long.sort_values("FECHA").head()

,FECHA,AÑO,MES,ESTACIÓN DEL AÑO,EVENTO INTERANUAL,estacion,TSM,Chla
0,1981-09-01,1981,Septiembre,Otoño,Neutro,Est 10°,28.6250,NaN
16,1981-09-01,1981,Septiembre,Otoño,Neutro,Est 9°,27.6583,NaN
15,1981-09-01,1981,Septiembre,Otoño,Neutro,Est 8°,29.9750,NaN
14,1981-09-01,1981,Septiembre,Otoño,Neutro,Est 7°,29.9917,NaN
13,1981-09-01,1981,Septiembre,Otoño,Neutro,Est 6°,30.2750,NaN


In [36]:
# Asignar nombres de columnas correctos
coords_df.columns = ['lon', 'estacion_dummy', 'lat']

# Crear nombres de estaciones estándar: Est 1°, Est 2°, ..., Est 17°
coords_df['estacion'] = [f'Est {i}°' for i in range(1, len(coords_df)+1)]

# Seleccionar columnas relevantes
coords_df = coords_df[['estacion', 'lon', 'lat']]

In [37]:
coords_df

,estacion,lon,lat
0,Est 1°,-109.3,23.8
1,Est 2°,-109.7,24.2
2,Est 3°,-110.1,24.6
3,Est 4°,-110.5,24.8
4,Est 5°,-110.8,25.3
5,Est 6°,-111.1,25.8
6,Est 7°,-111.2,26.3
7,Est 8°,-111.5,26.8
8,Est 9°,-111.9,27.2
9,Est 10°,-112.3,27.6


In [39]:
# Asegurarse de que los nombres de estación coincidan exactamente
coords_df.columns = coords_df.columns.str.strip()
coords_df['estacion'] = coords_df['estacion'].str.strip()

# Hacer merge con el dataset largo
data_geo = pd.merge(data_long, coords_df, on="estacion", how="left")

# Revisar si alguna estación no obtuvo coordenadas
missing_coords = data_geo[data_geo["lon"].isna()]["estacion"].unique()
if len(missing_coords) > 0:
    print("Estaciones sin coordenadas:", missing_coords)
else:
    print("Todas las estaciones tienen coordenadas.")

# Vista previa del DataFrame enriquecido
data_geo.tail()

Todas las estaciones tienen coordenadas.


,FECHA,AÑO,MES,ESTACIÓN DEL AÑO,EVENTO INTERANUAL,estacion,TSM,Chla,lon,lat
7577,2018-10-01,2018,Octubre,Otoño,Niño,Est 5°,29.0756,0.2772,-110.8,25.3
7578,2018-10-01,2018,Octubre,Otoño,Niño,Est 6°,28.6689,0.3656,-111.1,25.8
7579,2018-10-01,2018,Octubre,Otoño,Niño,Est 7°,28.7222,0.5083,-111.2,26.3
7580,2018-10-01,2018,Octubre,Otoño,Niño,Est 8°,28.4456,0.5119,-111.5,26.8
7581,2018-10-01,2018,Octubre,Otoño,Niño,Est 9°,27.9233,0.9144,-111.9,27.2


In [49]:
# Eliminar filas que no tienen años válidos en la columna "Year"
oni_df_clean = oni_df_raw[oni_df_raw["Year"].str.isnumeric()].copy()

# Convertir año y valores a tipos numéricos
oni_df_clean["Year"] = oni_df_clean["Year"].astype(int)
for col in oni_df_clean.columns[1:]:
    oni_df_clean[col] = pd.to_numeric(oni_df_clean[col], errors="coerce")

# Expansión a meses individuales
trimester_to_months = {
    "DJF": [(0, 12), (1, 1), (1, 2)],
    "JFM": [(1, 1), (1, 2), (1, 3)],
    "FMA": [(1, 2), (1, 3), (1, 4)],
    "MAM": [(1, 3), (1, 4), (1, 5)],
    "AMJ": [(1, 4), (1, 5), (1, 6)],
    "MJJ": [(1, 5), (1, 6), (1, 7)],
    "JJA": [(1, 6), (1, 7), (1, 8)],
    "JAS": [(1, 7), (1, 8), (1, 9)],
    "ASO": [(1, 8), (1, 9), (1, 10)],
    "SON": [(1, 9), (1, 10), (1, 11)],
    "OND": [(1, 10), (1, 11), (1, 12)],
    "NDJ": [(1, 11), (1, 12), (2, 1)],
}

oni_expanded = []

for _, row in oni_df_clean.iterrows():
    year = int(row["Year"])
    for trimester, months in trimester_to_months.items():
        oni_value = row[trimester]
        for offset, month in months:
            oni_year = year + (offset - 1)
            oni_expanded.append({"AÑO": oni_year, "MES_NUM": month, "ONI": oni_value})

oni_monthly_df = pd.DataFrame(oni_expanded).drop_duplicates(subset=["AÑO", "MES_NUM"])

# Clasificar ENSO
def clasificar_enso(val):
    if val >= 0.5:
        return "Niño"
    elif val <= -0.5:
        return "Niña"
    else:
        return "Neutro"

oni_monthly_df["ONI"] = pd.to_numeric(oni_monthly_df["ONI"], errors="coerce")
oni_monthly_df["ENSO_CLASE"] = oni_monthly_df["ONI"].apply(clasificar_enso)


# Mapeo manual de meses en español a número
meses_esp_map = {
    "Enero": 1, "Febrero": 2, "Marzo": 3, "Abril": 4,
    "Mayo": 5, "Junio": 6, "Julio": 7, "Agosto": 8,
    "Septiembre": 9, "Octubre": 10, "Noviembre": 11, "Diciembre": 12
}

# Aplicar el mapeo
data_geo["MES_NUM"] = data_geo["MES"].map(meses_esp_map)

# Verificar si hay valores nulos
print("Valores sin MES_NUM asignado:", data_geo["MES_NUM"].isna().sum())

# Volver a unir con oni_monthly_df
data_final = pd.merge(data_geo, oni_monthly_df, on=["AÑO", "MES_NUM"], how="left")

# Vista para confirmar
data_final.tail(12)

Valores sin MES_NUM asignado: 0


,FECHA,AÑO,MES,ESTACIÓN DEL AÑO,EVENTO INTERANUAL,estacion,TSM,Chla,lon,lat,MES_NUM,ONI,ENSO_CLASE
7570,2018-10-01,2018,Octubre,Otoño,Niño,Est 15°,27.5944,1.2078,-114.0,29.7,10,0.5,Niño
7571,2018-10-01,2018,Octubre,Otoño,Niño,Est 16°,27.7656,1.5967,-114.5,30.2,10,0.5,Niño
7572,2018-10-01,2018,Octubre,Otoño,Niño,Est 17°,28.0322,1.7532,-114.5,30.7,10,0.5,Niño
7573,2018-10-01,2018,Octubre,Otoño,Niño,Est 1°,29.9133,0.1522,-109.3,23.8,10,0.5,Niño
7574,2018-10-01,2018,Octubre,Otoño,Niño,Est 2°,29.7672,0.1781,-109.7,24.2,10,0.5,Niño
7575,2018-10-01,2018,Octubre,Otoño,Niño,Est 3°,29.5744,0.2155,-110.1,24.6,10,0.5,Niño
7576,2018-10-01,2018,Octubre,Otoño,Niño,Est 4°,29.5261,0.2304,-110.5,24.8,10,0.5,Niño
7577,2018-10-01,2018,Octubre,Otoño,Niño,Est 5°,29.0756,0.2772,-110.8,25.3,10,0.5,Niño
7578,2018-10-01,2018,Octubre,Otoño,Niño,Est 6°,28.6689,0.3656,-111.1,25.8,10,0.5,Niño
7579,2018-10-01,2018,Octubre,Otoño,Niño,Est 7°,28.7222,0.5083,-111.2,26.3,10,0.5,Niño


In [52]:
data_final.to_csv(r'C:\Users\Lenovo\Desktop\MCD\2025-1\Estadistica\ae-oceanografia-golfo-cali\data\interim\AABR_Notebooks\data_final.csv', index=False)


Clustering